In [1]:
!pip install -q langchain langchain-community langchain-huggingface chromadb ragatouille chonkie transformers accelerate bitsandbytes sentencepiece safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 101.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 137.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 40.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 35.8 MB/s eta 0:00:00
 

In [4]:
!nvidia-smi

Thu Feb 12 16:35:21 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# Load Dataset:
from datasets import load_dataset

data = load_dataset("m-ric/huggingface_doc")
print(f"Number of documents: {len(data['train'])}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

Number of documents: 2647


In [6]:
# print sample document:
doc = data["train"][0]

for key, val in doc.items():
    if isinstance(val, str) and len(val) > 500:
        print(f"{key} , {val[:500]}...")
    else:
        print(f"{key}. {val}")

text ,  Create an Endpoint

After your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. 

## 1. Enter the Hugging Face Repository ID and your desired endpoint name:

<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-docu...
source. huggingface/hf-endpoints-documentation/blob/main/docs/source/guides/create_endpoint.mdx


In [7]:
# convert text to chunks:
from chonkie import TokenChunker
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("gpt2")
chunker = TokenChunker(tokenizer=tokenizer, chunk_size=256, chunk_overlap=32)

chunks = []
for doc in data["train"]:
    chunk = chunker.chunk(doc["text"])
    chunks.append(chunk)

print(f"Total # of chunk-lists: {len(chunks)}")

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Total # of chunk-lists: 2647


In [8]:
ctexts = [chunk[0].text for chunk in chunks]
print(f"# of chunks to embed:{len(ctexts)}")

# of chunks to embed:2647


In [9]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata

embeddings = HuggingFaceEmbeddings(model="BAAI/bge-base-en-v1.5")
print(
    "Embedding model loaded!(bi-encoder that processes queries and documents together!)"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded!(bi-encoder that processes queries and documents together!)


In [10]:
from langchain_core.documents import Document

documents = []
for i, text in enumerate(ctexts):
    documents.append(
        Document(
            page_content=text, metadata={"chunk_id": i, "source": "huggingface_doc"}
        )
    )

docs = filter_complex_metadata(documents)
print(f"Prepared {len(docs)} documents for indexing.")

Prepared 2647 documents for indexing.


In [11]:
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="huggingface_docs_advanced",
)

print("Vector store created with HNSW indexing")
print(f"Total documents indexed: {vectorstore._collection.count()}")

Vector store created with HNSW indexing
Total documents indexed: 2647


In [12]:
# create retriver - bi-encoder for fast retrival!

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 20})

print("Retriever configured:")
print("  - Search type: similarity (cosine distance)")
print("  - Initial candidates: 20")
print("  - Algorithm: HNSW (logarithmic search time)")

Retriever configured:
  - Search type: similarity (cosine distance)
  - Initial candidates: 20
  - Algorithm: HNSW (logarithmic search time)


In [13]:
import time

# Test query
test_query = "How do I load a pretrained model?"

# Measure search time with HNSW
start_time = time.time()
results = retriever.invoke(test_query)
search_time = time.time() - start_time

print(f"Query: '{test_query}'")
print(f"Search time with HNSW: {search_time*1000:.2f} ms")
print(f"Documents retrieved: {len(results)}\n")

print("=" * 80)
print("Top 3 Results:")
print("=" * 80)
for i, doc in enumerate(results[:3], 1):
    print(f"\nResult {i}:")
    print(f"{doc.page_content[:200]}...")
    print(f"Source: {doc.metadata['source']}, Chunk ID: {doc.metadata['chunk_id']}")

Query: 'How do I load a pretrained model?'
Search time with HNSW: 83.48 ms
Documents retrieved: 20

Top 3 Results:

Result 1:
 Quickstart

This quickstart is intended for developers who are ready to dive into the code and see an example of how to integrate `timm` into their model training workflow.

First, you'll need to ins...
Source: huggingface_doc, Chunk ID: 1474

Result 2:
FrameworkSwitchCourse {fw} />

# Fine-tuning a model with the Trainer API[[fine-tuning-a-model-with-the-trainer-api]]

<CourseFloatingBanner chapter={3}
  classNames="absolute z-10 right-0 top-0"
  no...
Source: huggingface_doc, Chunk ID: 2398

Result 3:
FrameworkSwitchCourse {fw} />

# Introduction[[introduction]]

<CourseFloatingBanner
    chapter={3}
    classNames="absolute z-10 right-0 top-0"
/>

In [Chapter 2](/course/chapter2) we explored how t...
Source: huggingface_doc, Chunk ID: 2215


## HYDE


In [14]:
# load qwen2.5-7b-instruct:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

mid = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


tokenizer = AutoTokenizer.from_pretrained(mid, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    mid, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)

model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm

In [15]:
prompt = """Write a detailed, factual answer.
Do not mention uncertainty.

Question:
What is vector similarity search in information retrieval?
"""

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}], add_generation_prompt=True, tokenize=False
)
print(f"untokenized chat template:\n{inputs}")
inputs = tokenizer(inputs, return_tensors="pt")["input_ids"].to(model.device)
print(f"tokenized chat template:\n{inputs}")


with torch.no_grad():
    outputs = model.generate(
        inputs, max_new_tokens=300, temperature=0.2, top_p=0.9, do_sample=True
    )
print(f"Generated text:{tokenizer.decode(outputs[0] , skip_special_tokens= True)}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


untokenized chat template:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Write a detailed, factual answer.
Do not mention uncertainty.

Question:
What is vector similarity search in information retrieval?
<|im_end|>
<|im_start|>assistant

tokenized chat template:
tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   7985,    264,  11682,
             11,  59901,   4226,    624,   5404,    537,   6286,  26826,    382,
          14582,    510,   3838,    374,   4621,  37623,   2711,    304,   1995,
          56370,   5267, 151645,    198, 151644,  77091,    198]],
       device='cuda:0')
Generated text:system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Write a detailed, factual answer.
Do not mention uncertainty.

Question:
Wh

In [16]:
## function for hyde search:
def hyde_gen_doc(query, model, tokenizer):
    inputs = tokenizer.apply_chat_template(
        [{"role": "user", "content": query}],
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs, max_new_tokens=300, temperature=0.2, top_p=0.9, do_sample=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def hyde_search(query, model, tokenizer, retriever):
    hdoc = hyde_gen_doc(query, model, tokenizer)
    results = retriever.invoke(hdoc)
    return results


retrieved_docs = hyde_search(prompt, model, tokenizer, retriever)
for i, doc in enumerate(retrieved_docs):
    print(f"\nResult {i}:")
    print(f"{doc.page_content[:200]}...")
    print(f"Source: {doc.metadata['source']}, Chunk ID: {doc.metadata['chunk_id']}")


Result 0:
ext embeddings and semantic search. In this video we’ll explore how Transformer models represent text as embedding vectors and how these vectors can be used to find similar documents in a corpus. Text...
Source: huggingface_doc, Chunk ID: 1314

Result 1:
--
title: Image Similarity with Hugging Face Datasets and Transformers
thumbnail: /blog/assets/image_similarity/thumbnail.png
authors:
- user: sayakpaul
---

# Image Similarity with Hugging Face Datas...
Source: huggingface_doc, Chunk ID: 2226

Result 2:
 Search index

[FAISS](https://github.com/facebookresearch/faiss) and [Elasticsearch](https://www.elastic.co/elasticsearch/) enables searching for examples in a dataset. This can be useful when you wa...
Source: huggingface_doc, Chunk ID: 1172

Result 3:
--
title: "Retrieval Augmented Generation with Huggingface Transformers and Ray"
thumbnail: /blog/assets/12_ray_rag/ray_arch_updated.png
authors:
- user: ray-project
  guest: true
---

# Retrieval Aug...
Source: huggingface_

In [17]:
from sentence_transformers import CrossEncoder

# Load a cross-encoder model for reranking
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", device="cuda")

print("Cross-encoder reranker loaded")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Cross-encoder reranker loaded


In [18]:
def rerank_docs(query, documents, reranker, k=5):
    # make pairs
    pairs = [(query, doc) for doc in documents]

    # get scores and merge with documents:
    scores = reranker.predict(pairs)
    rdocs = [(doc, score) for doc, score in zip(documents, scores)]

    # rank documents according to scores and return:

    print(f"{rdocs}")
    rdocs.sort(key=lambda x: x[1], reverse=True)
    # return top-k documents:
    results = []
    for doc, score in rdocs[:k]:
        results.append({"content": doc, "score": float(score)})
    return results

In [19]:
# return cross ranked documents:
bidocs = [doc.page_content for doc in retrieved_docs]
rerank_docs(prompt, bidocs, reranker)

[("ext embeddings and semantic search. In this video we’ll explore how Transformer models represent text as embedding vectors and how these vectors can be used to find similar documents in a corpus. Text embeddings are just a fancy way of saying that we can represent text as an array of numbers called a vector. To create these embeddings we usually use an encoder-based model like BERT. In this example, you can see how we feed three sentences to the encoder and get three vectors as the output. Reading the text, we can see that walking the dog seems to be most similar to walking the cat, but let's see if we can quantify this! The trick to do the comparison is to compute a similarity metric between each pair of embedding vectors. These vectors usually live in a high-dimensional space, so a similarity metric can be anything that measures some sort of distance between vectors. One popular metric is cosine similarity, which uses the angle between two vectors to measure how close they are. In

[{'content': ' Search index\n\n[FAISS](https://github.com/facebookresearch/faiss) and [Elasticsearch](https://www.elastic.co/elasticsearch/) enables searching for examples in a dataset. This can be useful when you want to retrieve specific examples from a dataset that are relevant to your NLP task. For example, if you are working on a Open Domain Question Answering task, you may want to only return examples that are relevant to answering your question.\n\nThis guide will show you how to build an index for your dataset that will allow you to search it.\n\n## FAISS\n\nFAISS retrieves documents based on the similarity of their vector representations. In this example, you will generate the vector representations with the [DPR](https://huggingface.co/transformers/model_doc/dpr.html) model.\n\n1. Download the DPR model from 🤗 Transformers:\n\n```py\n>>> from transformers import DPRContextEncoder, DPRContextEncoderTokenizer\n>>> import torch\n>>> torch.set_grad_enabled(False)\n>>> ctx_encoder